## Starting Assumptions

We're going to model the cost of one specific home, and compare it to an equivalent rent. [The house is here](http://www.zillow.com/homes/for_sale/fsba_lt/24704738_zpid/12m_days/42.274213,-83.749015,42.251822,-83.794935_rect/14_zm/0_mmm/). I live in this area, and represent that the asking price, Zillow rent estimate, and financing terms are all reasonable.

So here's some rent estimates:

In [1]:
rent_per_month = 1650
rent_increase_per_month = 50.0 / 12

Zillow has also helpfully calculated the 30-year fixed mortgage terms:

In [2]:
price = 232000
down = 46400
apr = 0.03286
term = 12 * 30
yearly_taxes = 3317

## MATH TIME

First, the fundamental law of finance. Net present value of any sum of money:

In [3]:
def npv(future_value, discount_rate, t):
    return future_value / (1 + discount_rate) ** t

Calculating the net present value of all rental costs is fairly simple. Assume we're paying one month's rent as a deposit, and discount each month's rent:

In [4]:
def rent(months, r):
    def cost_of_month(t):
        return rent_per_month + (rent_increase_per_month * t)
    deposit = rent_per_month
    deposit_back = npv(deposit, r, months)
    return -deposit + sum(-npv(cost_of_month(t), r, t) for t in xrange(0, months)) + deposit_back

So, after 10 years, we will pay:

In [5]:
rent(12 * 10, 0)

-227750.0

Sounds about right.

Calculating the cost of buying is obviously more involved. Let's start with some basic mortgage definitions:

In [6]:
class Mortgage:
    def __init__(self, down, total, payments, rate):
        self.down, self.total, self.payments, self.rate = down, total, payments, rate
        
    def monthly_payment(self):
        r = self.rate
        P = self.total - self.down
        N = self.payments
        # From https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula
        return (r * P)  / (1 - (1 + r) ** -N)
        
    def remaining(self, t):
        balance = self.total - self.down
        for _ in xrange(0, t):
            interest = balance * self.rate
            balance -= (self.monthly_payment() - interest)
        return max(0, balance)

And plug in the terms Zillow gave us:

In [7]:
mortgage = Mortgage(down, price, term, apr / 12)

Now let's do some basic sanity checking:

In [8]:
mortgage.monthly_payment()

811.4145714967989

In [9]:
mortgage.remaining(12 * 30)

0

In [10]:
mortgage.remaining(0)

185600

All looking good.

Now we can model the cost of buying. We need to pay:

* money down for the mortgage
* closing costs on purchase, including agent fees, inspection, etc
* closing costs on sale
* monthly maintenance, monthly mortgage
* taxes (there are actually some important tax deductions you can claim against mortgage interest and PMI, but we're going to ignore those for simplicity)

In [11]:
def buy_in_general(mortgage, buy_closing_costs, sale_closing_costs, maintenance, taxes, months, appreciation, r):
    cost_to_close = mortgage.down + (mortgage.total * buy_closing_costs)
    
    mortgage_cost = sum(npv(mortgage.monthly_payment(), r, t) for t in xrange(0, min(months, mortgage.payments)))
    maintenance = sum(npv(maintenance * mortgage.total / 12, r, t) for t in xrange(0, months))
    
    taxes = sum(npv(taxes, r, year * 12) for year in xrange(months / 12))
    
    owed_bank = mortgage.remaining(months)
    from_buyer = mortgage.total * appreciation
    cash_at_sale = from_buyer - owed_bank - (from_buyer * sale_closing_costs)
    
    return -cost_to_close - mortgage_cost - maintenance - taxes + npv(cash_at_sale, r, months)

Let's assume:

* Closing costs are 1% on buying, and 6.5% on sale. Costs are dramatically higher on sale because the sale transaction is typically structured so that agent commissions are "paid" by the seller out of the price of the home.
* Maintenance is 1% per year.
* The home does not appreciate in value whatsoever during the term of the investment (which is arguably pessimistic).

In [12]:
def buy(months, r): 
    return buy_in_general(mortgage, 0.01, 0.065, 0.01, yearly_taxes, months, 1.0, r)

Let's see how much this house will cost us, undiscounted, over several years:

In [13]:
[buy(y * 12, 0) for y in xrange(1, 11)]

[-29080.519055983226,
 -40637.826478991454,
 -52067.81199742148,
 -63366.22822329373,
 -74528.68607812602,
 -85550.65006621709,
 -96427.43339024979,
 -107154.19290395285,
 -117725.92389638707,
 -128137.45470223753]

Let's compare that to renting:

In [14]:
[rent(y * 12, 0) for y in xrange(1, 11)]

[-20075.0,
 -40750.0,
 -62025.0,
 -83900.0,
 -106375.0,
 -129450.0,
 -153125.0,
 -177400.0,
 -202275.0,
 -227750.0]

So, renting becomes more expensive quickly, if we assume our money has no time value. That's not a good assumption, so let's delve a little deeper.

## IRR

IRR is a common concept when evaluating investments. The IRR essentially states the smallest time value your money needs to have for you to lose money on your investment. In other words, you want to know the rate that causes your discounted cash flow to become negative.

In [15]:
def irr(dcf):
    if dcf(0) < 0: return
    for ix in xrange(1000):
        r_t = float(ix) / 1000
        if dcf(r_t) < 0:
            return r_t

Note that there are cases where there is **no** IRR. Your investment loses money, even assuming your upfront dollars are valued equally to later hypothetical dollars. It cannot grow your wealth.

There will be cases where this happens with our house investment.

We can model this "investment" by subtracting the rent we're not paying from the other costs that we now must pay. Now, (drumroll please), let's look at the IRR for our hypothetical house, depending on how long we stay in it:

In [16]:
for year in xrange(1, 16):
    def dcf(r_i): return buy(12 * year, r_i / 12) - rent(12 * year, r_i / 12)
    print year, irr(dcf)

1 None
2 0.002
3 0.073
4 0.107
5 0.127
6 0.139
7 0.147
8 0.153
9 0.157
10 0.16
11 0.163
12 0.164
13 0.166
14 0.167
15 0.167


Not bad. We're well above the paltry [1.3%](http://www.bankrate.com/rates/interest-rates/treasury.aspx) current risk free rate in year three. In year five, our return rises above the [10%](http://www.investopedia.com/ask/answers/042415/what-average-annual-return-sp-500.asp) average annual returns for the S&P 500.

There's obviously a couple caveats. First, all of this analysis is predicated on the house being a principle residence. There are many benefits, especially regarding capital gains, that flow from the government in this case. Investment properties are a whole 'nother ball of wax.

Finally, let's keep in mind that this investment scenario has one big caveat: we disappear into thin air after we sell the house, and never spend money on housing again. We'll return to that assumption later.

## Bring on the Bears

Of course, market conditions can change drastically and quickly. What does our investment look like in a super-pessimistic case, where our house loses half its value?

In [17]:
def buy_ohno(months, r):
    return buy_in_general(mortgage, 0.02, 0.065, 0.01, yearly_taxes, months, 0.5, r)

In [18]:
for year in xrange(1, 21):
    def dcf(r_i): return buy_ohno(12 * year, r_i / 12) - rent(12 * year, r_i / 12)
    print year, irr(dcf)

1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
11 0.024
12 0.07
13 0.096
14 0.113
15 0.125
16 0.133
17 0.14
18 0.144
19 0.148
20 0.151


Pretty grim, but still not a doomsday. We'll need to stay in the house for a long time to beat the market if we get caught in a bad market cycle.

## The Long View

We've established that the longer you stay in the house, the better off you are. Let's look at what happens after your mortgage is payed off, and if your house reliable appreciates to keep up with a 2% inflation rate over the years.

In [19]:
for year in xrange(1, 31, 5):
    def buy_and_appreciate(months, r): 
        return buy_in_general(mortgage, 0.01, 0.065, 0.01, yearly_taxes, months, 1.02 ** year, r)
    def dcf(r_i): return buy_and_appreciate(12 * year, r_i / 12) - rent(12 * year, r_i / 12)
    print year, irr(dcf)

1 None
6 0.187
11 0.187
16 0.181
21 0.176
26 0.173


What? Our rate of return gets **worse** the longer we wait to sell?

What's happening here is we're forgetting that we actually have to live somewhere if we don't own a house. Let's do a more realistic comparison: whenever we sell our house, we'll buy another similar house somewhere else. We'll also plow all of our equity from before into the new mortgage.

In other words, our **very** simplistic assumption is that we essentially have the same mortgage all along. We just need to pay both buying and selling closing costs each time we switch houses.

In [20]:
for year in (1, 2, 5, 10, 25, 50):
    swap_months = year * 12
    total_months = 50 * 12
    swaps = [m for m in xrange(swap_months, total_months, swap_months)]

    def buy_and_appreciate(r): 
        return buy_in_general(mortgage, 0.01, 0.065, 0.01, yearly_taxes, total_months, 1.0, r)

    def dcf_swapping_houses(r):
        switching_costs = sum(npv(0.075 * mortgage.total, m, r / 12) for m in swaps)
        return buy_and_appreciate(r / 12) - rent(total_months, r / 12) - switching_costs

    print year, irr(dcf_swapping_houses)

1 0.013
2 0.035
5 0.067
10 0.097
25 0.139
50 0.169


So it looks like we should plan on switching houses every 10+ years if we want to beat the market. If we're planning on staying put for a very long time, a house looks like a fantastic investment.

Another thing to keep in mind: with this time horizon, the price of the house itself is practically inconsequential. Here's the present value of the cash we'll get on sale, discounted at 10% per year:

In [21]:
npv(mortgage.total, 0.1 / 12, 50 * 12)

1595.9284749821113

This is a tiny fraction of the total NPV of our housing costs:

In [22]:
buy(50 * 12, 0.1 / 12)

-198445.4001645193

## TL;DR

So is housing a shitty investment? It certainly can be, especially if you change houses every year. But if you're in a specific location for the long haul, investment in housing for your principle residence will probably beat market returns. Just plan to stay in it for a long time.

Another final note: the current lending environment is **very** favorable towards buying a house. Interest rates are at all time lows. All the analysis here will certainly need to change in a future lending environment.